<h1 align=center><font size = 6>Natural Language Processing using TensorFlow</font></h1>

Done by
Firda Anindita Latifah</p>
firdaaninditalatifah@gmail.com</p>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/Data

In [ ]:
import pandas as pd

df = pd.read_csv('Corona_NLP_train.csv', encoding="Latin-1")
df = df[['OriginalTweet','Sentiment']]

In [ ]:
for i in range(0,len(df)):
    if(df['Sentiment'][i]=='Extremely Negative'):
        df['Sentiment'][i]='Negative'
    elif(df['Sentiment'][i]=='Extremely Positive'):
        df['Sentiment'][i]='Positive'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(x=df.Sentiment)
plt.title('The number of elements in each class in the data')
plt.tight_layout()
plt.show()

In [ ]:
category = pd.get_dummies(df.Sentiment)
df_new = pd.concat([df, category], axis=1)
df_new = df_new.drop(columns='Sentiment')
df_new

In [ ]:
import re
from string import punctuation, digits
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def remove_punctuation(s):
    list_punctuation = list(punctuation)
    for i in list_punctuation:
        s = s.replace(i,'')
    return s.lower()

def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', '', sentence)
    sentence = re.sub(r"\@(\w+)", '', sentence) 
    sentence = re.sub(r"\#(\w+)", '', sentence) 
    sentence = re.sub(r"\$(\w+)", '', sentence) 
    sentence= re.sub(r'[^\w]', ' ', sentence)
    sentence= re.sub(r"[&amp]", '', sentence)
    sentence = sentence.replace("-",' ')
    tokens = sentence.split()
    tokens = [remove_punctuation(w) for w in tokens] 
    stop_words = set(stopwords.words('english')) 
    tokens = [w for w in tokens if not w in stop_words]
    remove_digits = str.maketrans('', '', digits)
    tokens = [w.translate(remove_digits) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
    tokens = ' '.join(tokens)
    return tokens

In [ ]:
df_new['OriginalTweet'] = df_new['OriginalTweet'].apply(lambda sentence:clean_sentence(sentence))

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator

allWords = ''.join([twts for twts in df_new['OriginalTweet']])
wordCloud = WordCloud(width = 300,height =150,random_state = 21,max_font_size =119).generate(allWords)
plt.imshow(wordCloud,interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(de_punct_sent):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(de_punct_sent))
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='x')
tokenizer.fit_on_texts(df_baru['OriginalTweet'].values)
df_token = tokenizer.texts_to_sequences(df_baru['OriginalTweet'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
lens =  [len(s) for s in df_token]

plt.title('The maximum number of words in a sentence')
plt.xlabel('The max number of words in a sentence')
plt.ylabel('The number of sentences')
plt.hist(lens,bins=200)
plt.show()

In [ ]:
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 40
df_train = pad_sequences(df_token, maxlen=maxlen, padding='post', truncating='post') 

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=25000, output_dim=16),
    tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
class Call(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}): 
    if(logs.get('accuracy') > 0.85 and logs.get('val_accuracy') > 0.85):
      print("\nAccuracy > 85%") 
      self.model.stop_training = True 
 
callbacks = Call()

In [ ]:
from sklearn.model_selection import train_test_split

label = df_new[['Negative',	'Neutral',	'Positive']].values
train_data, test_data, train_label, test_label = train_test_split(df_train, label, test_size = 0.2, random_state = 42)
print(train_data.shape,train_label.shape)
print(test_data.shape,test_label.shape)

In [ ]:
history = model.fit(train_data, train_label, epochs=10, 
                    validation_data =(test_data, test_label), verbose=2, callbacks=[callbacks])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss')
plt.legend()

plt.show()